In [1]:
import sys
sys.path.append("..")

from qiskit.providers.fake_provider import Fake27QPulseV1, Fake5QV1, Fake20QV1
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from src.state_of_the_art.sabre import run_sabre
from src.state_of_the_art.cirq import run_cirq,from_backend_to_edges
from src.state_of_the_art.pytket import run_pyket
from src.state_of_the_art.qmap import run_qmap
from src.isl_routing.mapping.routing import POLY_QMAP
from src.isl_routing.utils.isl_data_loader import *
from src.isl_routing.utils.circuit_utils import *
from src.visiualisation.plots import *
from time import time
import warnings
warnings.filterwarnings("ignore")


In [2]:
edges = FakeSherbrooke().configuration().coupling_map

In [3]:
file_path = f"../benchmarks/polyhedral/queko-bss-16qbt/16QBT_{500}CYC_QSE_{0}.json"
data = json_file_to_isl(file_path) 

In [4]:
poly_mapper = POLY_QMAP(edges, data)
poly_mapper.run(heuristic_method="closure",initial_mapping_method="trivial")

poly_mapper.results[0]

{'swap_count': 894, 'circuit_depth': 1354}

In [5]:
print("correct swaps :",poly_mapper.circuit.count_ops().get("swap", 0) ," correct depth: ", poly_mapper.circuit.depth())

correct swaps : 894  correct depth:  1354


In [6]:
sabre = run_sabre(data, edges,layout="trivial") 


print("swaps :",sabre["swap_count"])
print("depth :",sabre["circuit_depth"])

swaps : 879
depth : 1339


In [7]:
from qiskit.transpiler import CouplingMap

def is_routable(circuit, backend):

    coupling_map = backend.configuration().coupling_map
    cm = CouplingMap(coupling_map)
    
    for instr, qargs, _ in circuit.data:
        if instr.num_qubits > 1:

            q0 = qargs[0]._index
            q1 = qargs[1]._index

            if cm.distance(q0, q1) != 1:
                print(f"Gate {instr.name} on qubits {q0} and {q1} is not directly supported.")
                return False
    return True


routable = is_routable(poly_mapper.circuit, FakeSherbrooke())
if routable:
    print("The circuit is directly routable on the backend.")
else:
    print("The circuit is NOT directly routable on the backend.")


The circuit is directly routable on the backend.
